In [113]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

In [114]:
innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')
innerOut_df_origin['Timestamp'] = pd.to_datetime(innerOut_df_origin['Timestamp'])
innerOut_df_origin = innerOut_df_origin.sort_values(by = ['Timestamp'])

outKeyword_df_origin = pd.read_csv('../Data/Preprocessed/Keyword/out_keyword.csv')
outKeyword_df_origin.drop(columns = ['index'], inplace = True)

innerOut_df_origin.loc[(innerOut_df_origin['New IP'].isin(outKeyword_df_origin.loc[outKeyword_df_origin['category'] == 'firstOutWord', 'New IP'])), 'firstOutWord'] = True
innerOut_df_origin.loc[:, 'firstOutWord'] = innerOut_df_origin.loc[:, 'firstOutWord'].fillna(False)
innerOut_df_origin = innerOut_df_origin.loc[innerOut_df_origin['bigPath'] != '기타']
innerOut_df_origin['category_small'] = innerOut_df_origin.apply(lambda x : x['bigPath'] + '//' + x['middlePath'] + '//' + x['smallPath'], axis = 1)
innerOut_df_origin['category_middle'] = innerOut_df_origin.apply(lambda x : x['bigPath'] + '//' + x['middlePath'], axis = 1)

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21896\4041046657.py:1: DtypeWarning: Columns (4,5,6,7,20,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')
C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21896\4041046657.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  innerOut_df_origin.loc[(innerOut_df_origin['New IP'].isin(outKeyword_df_origin.loc[outKeyword_df_origin['category'] == 'firstOutWord', 'New IP'])), 'firstOutWord'] = True


In [ ]:
# 대분류 집계
innerOut_df = innerOut_df_origin.copy()
standard_column = ['bigPath', 'User Type']
groups = innerOut_df.groupby(standard_column)

tmp01 = groups.agg({'New IP' : 'nunique',
                    'URL' : 'count'}).reset_index()
tmp02 = innerOut_df.loc[innerOut_df['idx'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'idx_New IP',
                                                                                                                                      'URL' : 'idx_URL'})
tmp03 = innerOut_df.loc[innerOut_df['news_id'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                        'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'news_id_New IP',
                                                                                                                                          'URL' : 'news_id_URL'})
tmp_total = innerOut_df.groupby(['User Type']).agg({'New IP' : 'nunique',
                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'total_New IP',
                                                                                                      'URL' : 'total_URL'})

result = tmp01.merge(tmp02, on = standard_column, how = 'left').merge(tmp03, on = standard_column, how = 'left').merge(tmp_total, on = ['User Type'], how = 'left').fillna(0)
result['접속자 비율'] = round(result['New IP'] / result['total_New IP'] * 100, 2)
result['페이지 뷰 비율'] = round(result['URL'] / result['total_URL'] * 100, 2)
result['게시글 비율'] = round(result['idx_URL'] / result['URL'] * 100 , 2)
result['게시글 독자 비율'] = round(result['idx_New IP'] / result['New IP'] * 100, 2)
result['페이지 뷰 대비 뉴스 뷰 비율'] = round(result['news_id_URL'] / result['URL'] * 100, 2)
result['기사 독자 비율'] = round(result['news_id_New IP'] / result['New IP'] * 100, 2)
result['메뉴 페이지 뷰 비율'] = round((result['URL'] - result['news_id_URL'] - result['idx_URL']) / result['URL'] * 100, 2)
result = result.drop(columns = ['idx_New IP', 'idx_URL', 'news_id_New IP', 'news_id_URL', 'total_New IP', 'total_URL'])

In [154]:
result.loc[result['User Type'] == '내부'].sort_values(by = ['New IP', 'URL'], ascending = False)

,bigPath,User Type,New IP,URL,접속자 비율,페이지 뷰 비율,게시글 비율,게시글 독자 비율,페이지 뷰 대비 뉴스 뷰 비율,기사 독자 비율,메뉴 페이지 뷰 비율
18,채용/인재,내부,80111,428419,39.69,21.41,79.01,85.97,0.83,3.53,20.16
6,뉴스,내부,78121,421878,38.71,21.08,0.00,0.00,63.70,78.72,36.30
12,메인,내부,18421,40122,9.13,2.00,0.00,0.00,0.00,0.00,100.00
16,작품/조경수/자재,내부,18322,223696,9.08,11.18,52.79,83.05,0.48,3.24,46.73
14,인기통합정보,내부,17087,27228,8.47,1.36,0.00,0.00,69.40,90.89,30.60
4,고객센터,내부,12518,54255,6.20,2.71,0.00,0.00,0.00,0.00,100.00
20,커뮤니티,내부,12312,83882,6.10,4.19,36.91,52.01,5.26,21.91,57.83
10,매거진,내부,10816,110611,5.36,5.53,0.00,0.00,25.53,63.72,74.47
22,포토갤러리,내부,4942,530349,2.45,26.50,13.24,53.60,0.25,17.02,86.51
2,검색,내부,4064,47229,2.01,2.36,0.00,0.00,0.00,0.00,100.00


In [155]:
result.loc[result['User Type'] == '외부'].sort_values(by = ['New IP', 'URL'], ascending = False)

,bigPath,User Type,New IP,URL,접속자 비율,페이지 뷰 비율,게시글 비율,게시글 독자 비율,페이지 뷰 대비 뉴스 뷰 비율,기사 독자 비율,메뉴 페이지 뷰 비율
7,뉴스,외부,61088,110669,54.94,40.39,0.00,0.00,76.51,97.02,23.49
19,채용/인재,외부,12281,63422,11.04,23.15,62.08,77.69,2.76,12.33,35.16
21,커뮤니티,외부,11294,16289,10.16,5.95,83.99,95.73,2.84,3.21,13.17
17,작품/조경수/자재,외부,9493,18931,8.54,6.91,61.49,82.07,2.37,3.98,36.14
5,고객센터,외부,9405,29829,8.46,10.89,0.00,0.00,0.00,0.00,100.00
11,매거진,외부,8195,11516,7.37,4.20,0.00,0.00,85.85,97.28,14.15
3,검색,외부,5499,13128,4.95,4.79,0.00,0.00,0.00,0.00,100.00
15,인기통합정보,외부,2480,3080,2.23,1.12,0.00,0.00,90.78,96.98,9.22
13,메인,외부,1502,3583,1.35,1.31,0.00,0.00,0.00,0.00,100.00
23,포토갤러리,외부,813,2561,0.73,0.93,35.49,39.61,18.16,47.23,46.35


In [ ]:
# 대분류 집계
innerOut_df = innerOut_df_origin.copy()
standard_column = ['bigPath', 'User Type']
groups = innerOut_df.groupby(standard_column)

tmp01 = groups.agg({'New IP' : 'nunique',
                    'URL' : 'count'}).reset_index()
tmp02 = innerOut_df.loc[innerOut_df['idx'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'idx_New IP',
                                                                                                                                      'URL' : 'idx_URL'})
tmp03 = innerOut_df.loc[innerOut_df['news_id'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                        'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'news_id_New IP',
                                                                                                                                          'URL' : 'news_id_URL'})
tmp_total = innerOut_df.groupby(['User Type']).agg({'New IP' : 'nunique',
                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'total_New IP',
                                                                                                      'URL' : 'total_URL'})

result = tmp01.merge(tmp02, on = standard_column, how = 'left').merge(tmp03, on = standard_column, how = 'left').merge(tmp_total, on = ['User Type'], how = 'left').fillna(0)
result['접속자 비율'] = round(result['New IP'] / result['total_New IP'] * 100, 2)
result['페이지 뷰 비율'] = round(result['URL'] / result['total_URL'] * 100, 2)
result['게시글 비율'] = round(result['idx_URL'] / result['URL'] * 100 , 2)
result['게시글 독자 비율'] = round(result['idx_New IP'] / result['New IP'] * 100, 2)
result['페이지 뷰 대비 뉴스 뷰 비율'] = round(result['news_id_URL'] / result['URL'] * 100, 2)
result['기사 독자 비율'] = round(result['news_id_New IP'] / result['New IP'] * 100, 2)
result['메뉴 페이지 뷰 비율'] = round((result['URL'] - result['news_id_URL'] - result['idx_URL']) / result['URL'] * 100, 2)
result = result.drop(columns = ['idx_New IP', 'idx_URL', 'news_id_New IP', 'news_id_URL', 'total_New IP', 'total_URL'])

In [187]:
# 소분류 집계
innerOut_df = innerOut_df_origin.copy()
innerOut_df = innerOut_df.sort_values(by = 'Timestamp')
standard_column = ['User Type', 'bigPath', 'middlePath', 'smallPath']
groupby = innerOut_df.groupby(standard_column)

tmp01 = groupby.agg({'New IP' : 'nunique',
                     'URL' : 'count'}).reset_index()
tmp02 = innerOut_df.loc[innerOut_df['idx'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'idx_New IP',
                                                                                                                                      'URL' : 'idx_URL'})
tmp03 = innerOut_df.loc[innerOut_df['news_id'].notnull()].groupby(standard_column).agg({'New IP' : 'nunique',
                                                                                        'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'news_id_New IP',
                                                                                                                                          'URL' : 'news_id_URL'})

tmp_total = innerOut_df.groupby(['User Type']).agg({'New IP' : 'nunique',
                                                    'URL' : 'count'}).reset_index().rename(columns = {'New IP' : 'total_New IP',
                                                                                                      'URL' : 'total_URL'})

result = tmp01.merge(tmp02, on = standard_column, how = 'left').merge(tmp03, on = standard_column, how = 'left').merge(tmp_total, on = ['User Type'], how = 'left').fillna(0)
result['접속자 비율'] = round(result['New IP'] / result['total_New IP'] * 100, 2)
result['페이지 뷰 비율'] = round(result['URL'] / result['total_URL'] * 100, 2)
result['게시글 비율'] = round(result['idx_URL'] / result['URL'] * 100 , 2)
result['게시글 독자 비율'] = round(result['idx_New IP'] / result['New IP'] * 100, 2)
result['페이지 뷰 대비 뉴스 뷰 비율'] = round(result['news_id_URL'] / result['URL'] * 100, 2)
result['기사 독자 비율'] = round(result['news_id_New IP'] / result['New IP'] * 100, 2)
result['메뉴 페이지 뷰 비율'] = round((result['URL'] - result['news_id_URL'] - result['idx_URL']) / result['URL'] * 100, 2)
result = result.drop(columns = ['idx_New IP', 'idx_URL', 'news_id_New IP', 'news_id_URL', 'total_New IP', 'total_URL'])
result = result.sort_values(by = ['User Type', 'bigPath', 'New IP', 'URL'], ascending = False)
result

,User Type,bigPath,middlePath,smallPath,New IP,URL,접속자 비율,페이지 뷰 비율,게시글 비율,게시글 독자 비율,페이지 뷰 대비 뉴스 뷰 비율,기사 독자 비율,메뉴 페이지 뷰 비율
1183,외부,포토갤러리,조경사진홈,메인,422,536,0.38,0.20,3.36,3.79,81.72,84.60,14.93
1177,외부,포토갤러리,전체사진,전체사진메인,199,661,0.18,0.24,72.47,93.47,0.00,0.00,27.53
1217,외부,포토갤러리,휴식/전시시설,휴식/전시시설메인,40,137,0.04,0.05,21.17,42.50,0.00,0.00,78.83
1163,외부,포토갤러리,메인,메인,39,49,0.04,0.02,0.00,0.00,0.00,0.00,100.00
1133,외부,포토갤러리,error,error,35,97,0.03,0.04,98.97,100.00,0.00,0.00,1.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,내부,검색,설계소재,설계소재,67,1535,0.03,0.08,0.00,0.00,0.00,0.00,100.00
8,내부,검색,커뮤니티,커뮤니티,59,1545,0.03,0.08,0.00,0.00,0.00,0.00,100.00
6,내부,검색,조경사진,조경사진,42,1509,0.02,0.08,0.00,0.00,0.00,0.00,100.00
3,내부,검색,도움말,도움말,28,1489,0.01,0.07,0.00,0.00,0.00,0.00,100.00


In [190]:
inner = result.loc[result['User Type'] == '내부']
inner.groupby(['bigPath']).apply(lambda x : x.iloc[:3])

User Type    bigPath middlePath    smallPath  New IP     URL  \
bigPath                                                                      
e-아카데미    0          내부     e-아카데미     e-아카데미       e-아카데미     693    2790   
검색        1          내부         검색       검색메인         검색메인    3824   36070   
          2          내부         검색         뉴스           뉴스     188    1691   
          7          내부         검색       채용인재         채용인재     172    1747   
고객센터      10         내부       고객센터      고객센터홈        고객센터홈    7056   25165   
          11         내부       고객센터       공지사항       공지사항메인    4242    9769   
          13         내부       고객센터   문의/제안/신고   문의/제안/신고메인    1179    3643   
뉴스        53         내부         뉴스        뉴스홈          뉴스홈   61108  228309   
          80         내부         뉴스    전체/주요뉴스    전체/주요뉴스메인   14958  121785   
          69         내부         뉴스         메인           메인    4594    9967   
마이페이지     105        내부      마이페이지       조경뉴스   조경뉴스 내정보메인    1958   29093   
          110        내부      마이페이지      채용/인재       채용공고등록     315    1017   
          108        내부      마이페이지      채용/인재  채용/인재 내정보메인     291     629   
매거진       113        내부        매거진       매거진홈         매거진홈    6549   18412   
          149        내부        매거진       전체기사       전체기사메인    3497   56419   
          114        내부        매거진         메인           메인     654    2587   
메인        166        내부         메인         메인           메인   18421   40122   
인기통합정보    269        내부     인기통합정보     인터뷰/취재           뉴스    2371    3071   
          203        내부     인기통합정보   기획/특집/연재           뉴스    1721    1951   
          314        내부     인기통합정보      학술/교육        채용/인재    1317    1478   
작품/조경수/자재 395        내부  작품/조경수/자재       전체정보       전체정보메인    8377  110505   
          348        내부  작품/조경수/자재  설계/시공/재료홈    설계/시공/재료홈    7580   33168   
          399        내부  작품/조경수/자재     조경수/식물     조경수/식물메인    1323   16297   
채용/인재     410        내부      채용/인재  계획/설계/디자인           메인   31232  129557   
          490        내부      채용/인재       전체채용           메인   23948   92277   
          503        내부      채용/인재     채용/인재홈       채용/인재홈   15785   50815   
커뮤니티      547        내부       커뮤니티      커뮤니티홈        커뮤니티홈    4874   22177   
          515        내부       커뮤니티       자유토론       자유토론메인    3536    9430   
          521        내부       커뮤니티      조경인광장       조경인Q&A    1298    4924   
포토갤러리     599        내부      포토갤러리       전체사진       전체사진메인    2271   92380   
          607        내부      포토갤러리      조경사진홈           메인    1368    3717   
          647        내부      포토갤러리    휴식/전시시설    휴식/전시시설메인    1206   81508   

               접속자 비율  페이지 뷰 비율  게시글 비율  게시글 독자 비율  페이지 뷰 대비 뉴스 뷰 비율  \
bigPath                                                                
e-아카데미    0      0.34      0.14    9.25      27.85              8.39   
검색        1      1.89      1.80    0.00       0.00              0.00   
          2      0.09      0.08    0.00       0.00              0.00   
          7      0.09      0.09    0.00       0.00              0.00   
고객센터      10     3.50      1.26    0.00       0.00              0.00   
          11     2.10      0.49    0.00       0.00              0.00   
          13     0.58      0.18    0.00       0.00              0.00   
뉴스        53    30.28     11.41    0.00       0.00             81.59   
          80     7.41      6.09    0.00       0.00             59.43   
          69     2.28      0.50    0.00       0.00              0.00   
마이페이지     105    0.97      1.45    2.22      17.93              0.00   
          110    0.16      0.05   82.69      85.71              0.00   
          108    0.14      0.03    0.00       0.00              0.00   
매거진       113    3.24      0.92    0.00       0.00             78.51   
          149    1.73      2.82    0.00       0.00             14.39   
          114    0.32      0.13    0.00       0.00              0.00   
메인        166    9.13      2.00    0.00       0.00              0

In [191]:
outer = result.loc[result['User Type'] == '외부']
outer.groupby(['bigPath']).apply(lambda x : x.iloc[:3])

User Type    bigPath middlePath    smallPath  New IP    URL  \
bigPath                                                                      
e-아카데미    648         외부     e-아카데미     e-아카데미       e-아카데미      44     57   
검색        649         외부         검색       검색메인         검색메인    4698  11595   
          652         외부         검색        매거진          매거진     488    576   
          650         외부         검색         뉴스           뉴스     466    591   
고객센터      658         외부       고객센터      고객센터홈        고객센터홈    9292  29341   
          661         외부       고객센터   문의/제안/신고   문의/제안/신고메인      54     98   
          675         외부       고객센터       회원정보  회원정보/비밀번호변경      30     54   
뉴스        697         외부         뉴스        뉴스홈          뉴스홈   55178  86647   
          720         외부         뉴스    전체/주요뉴스    전체/주요뉴스메인    4331   9514   
          709         외부         뉴스         메인           메인    1560   3967   
마이페이지     743         외부      마이페이지       조경뉴스   조경뉴스 내정보메인     327    524   
          748         외부      마이페이지      채용/인재       채용공고등록      73    241   
          746         외부      마이페이지      채용/인재  채용/인재 내정보메인      56    135   
매거진       750         외부        매거진       매거진홈         매거진홈    6574   8686   
          753         외부        매거진     에코스케이프        매거진구매     427    522   
          795         외부        매거진      환경과조경         특별기획     405    454   
메인        798         외부         메인         메인           메인    1502   3583   
인기통합정보    889         외부     인기통합정보     인터뷰/취재           뉴스     238    291   
          924         외부     인기통합정보      학술/교육        채용/인재     175    200   
          834         외부     인기통합정보   기획/특집/연재         커뮤니티     144    161   
작품/조경수/자재 952         외부  작품/조경수/자재  설계/시공/재료홈    설계/시공/재료홈    3988   5755   
          997         외부  작품/조경수/자재       전체정보       전체정보메인    3088   5204   
          984         외부  작품/조경수/자재         자재         자재메인     559    896   
채용/인재     1012        외부      채용/인재  계획/설계/디자인           메인    4330  23820   
          1023        외부      채용/인재         메인           메인    4023   7047   
          1080        외부      채용/인재       전체채용           메인    3902  15470   
커뮤니티      1132        외부       커뮤니티      커뮤니티홈        커뮤니티홈    4362   5390   
          1108        외부       커뮤니티      조경인광장       조경인Q&A    2240   3256   
          1098        외부       커뮤니티        자료실         기타일반    1704   2391   
포토갤러리     1183        외부      포토갤러리      조경사진홈           메인     422    536   
          1177        외부      포토갤러리       전체사진       전체사진메인     199    661   
          1217        외부      포토갤러리    휴식/전시시설    휴식/전시시설메인      40    137   

                접속자 비율  페이지 뷰 비율  게시글 비율  게시글 독자 비율  페이지 뷰 대비 뉴스 뷰 비율  \
bigPath                                                                 
e-아카데미    648     0.04      0.02   42.11      36.36             43.86   
검색        649     4.22      4.23    0.00       0.00              0.00   
          652     0.44      0.21    0.00       0.00              0.00   
          650     0.42      0.22    0.00       0.00              0.00   
고객센터      658     8.36     10.71    0.00       0.00              0.00   
          661     0.05      0.04    0.00       0.00              0.00   
          675     0.03      0.02    0.00       0.00              0.00   
뉴스        697    49.62     31.62    0.00       0.00             86.33   
          720     3.89      3.47    0.00       0.00             61.27   
          709     1.40      1.45    0.00       0.00              0.00   
마이페이지     743     0.29      0.19   22.33      18.96              0.00   
          748     0.07      0.09   84.65      95.89              0.00   
          746     0.05      0.05    0.00       0.00              0.00   
매거진       750     5.91      3.17    0.00       0.00             92.78   
          753     0.38      0.19    0.00       0.00             91.57   
          795     0.36      0.17    0.00       0.00             95.81   
메인        798     1.35      1.31    0.00       0